In [1]:

import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz
from knowledge_propagation.utils.eval import is_significantly_different
from collections import Counter
from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()
from typing import List, Dict, Tuple

os.getcwd()
def macro_averaging(df: pd.DataFrame, metrics: List[str], multi_level_averaging: List[str]):
    """
    Do macro-averaging over the given metrics and multi-level averaging categories.
    """
    extracted_multi_level_cols = [[m, "mean"] for m in metrics]
    while len(multi_level_averaging) > 0:
        # first take the mean over each generation,
        # and, only take `mean` of `rouge1` and  `llm_accuracy` column groups
        df_over_cols = df.groupby(multi_level_averaging, observed=True).describe()[extracted_multi_level_cols]
        # remove the multi-level column indices, since there's only one sub-level -- "mean"
        df_over_cols.columns = df_over_cols.columns.get_level_values(0)

        # reset index to flatten the multi-level column indices for the next macro-averaging class
        df = df_over_cols.reset_index(inplace=False)
        multi_level_averaging.pop(-1)
    return df

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fpath = "/data/users/zliu/mend/synstory_exp_output/4K_heavy_share_layer4_15/syn_story/mend_eval_loss=clm_input=seen_n=350_prompt=no_w-gen_wo-icl_4K_test_ood-entity-question.xlsx"
df1 = pd.read_excel(fpath)
# df2 = pd.read_excel("/u/zliu/datastor1/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_id/base_n=500_prompt=no_w-gen_wo-icl_ice=True.xlsx")

In [93]:
# 4K_test_id
# val_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/test_text_data_id_entity152_rel31.jsonl")
# 4K_test_ood-entity
# val_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/test_text_data_ood-entity_entity37_rel31.jsonl")
# 4K_test_ood-relation
# val_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/test_text_data_ood-relation_entity152_rel7.jsonl")
# 4K_test_ood
val_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/test_text_data_ood_entity37_rel7.jsonl")

questions = []
for datum in val_data:
    for question_type, test_queries in [("efficacy", datum["questions"]),]:
        for question_key in ["alias_question", "unalias_question"][:]:
            for q_i, test_query in enumerate(test_queries):
                questions.append(test_query[question_key])

In [94]:
df1["question"] = questions

In [95]:
df2 = pd.read_excel("/u/zliu/datastor1/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")
(df1["question"] == df2["question"]).all()

np.True_

In [12]:
df1.iloc[2733]["edit_input"]

'[[<|begin_of_text|>Orange Strategies LLC drew early inspiration from The Haitian Revolution to shape its culture. Over time, English Civil War became a common point of reflection within the company. Later, it highlighted Napoleonic Wars in an initiative promoting historical awareness.<|end_of_text|>]]'

In [19]:
df1[df1["llm_accuracy"] == 0].iloc[1]["question"]

'What type of organism is the species that Purple Marketing Ltd. supported a conservation project for?'

In [20]:
len(df1[df1["llm_accuracy"] == 0])

960

In [25]:
df1[df1["llm_accuracy"] == 0]

,id,edit_input,stage,question_key,question_type,question,answer,predicted_answer_idx,predicted_answer,llm_accuracy
1,0,[[<|begin_of_text|>Purple Marketing Ltd. devel...,post-edit,alias_question,efficacy,What is the diet of the species that Purple Ma...,Insects and small invertebrates,0,Monkeys,0.0
2,0,[[<|begin_of_text|>Purple Marketing Ltd. devel...,post-edit,alias_question,efficacy,What type of organism is the species that Purp...,Crustacean,0,Mammals,0.0
7,1,[[<|begin_of_text|>Evelyn Hall became fascinat...,post-edit,alias_question,efficacy,What is the diet of the species that triggered...,"Leaves, twigs, and fruits of trees and shrubs",0,"Carnivorous; eats small mammals, birds, carrio...",0.0
13,2,[[<|begin_of_text|>David Garcia developed a pa...,post-edit,alias_question,efficacy,Who was the most important leader or figure in...,Maximilien Robespierre,0,Napoleon Bonaparte,0.0
16,3,[[<|begin_of_text|>Teal Manufacturing Ltd. beg...,post-edit,alias_question,efficacy,What writing system is used by the language th...,Cyrillic,0,Latin alphabet,0.0
...,...,...,...,...,...,...,...,...,...,...
2722,347,[[<|begin_of_text|>Hill Energy Corp. built its...,post-edit,alias_question,efficacy,What is the genre or style of the creative wor...,"Romantic novel, social satire",0,"Crime, black comedy, thriller",0.0
2725,347,[[<|begin_of_text|>Hill Energy Corp. built its...,post-edit,unalias_question,efficacy,Where was Pan's Labyrinth produced or created?,Spain,0,United Kingdom,0.0
2729,348,[[<|begin_of_text|>Caleb Diaz developed a pass...,post-edit,alias_question,efficacy,Who was the most important leader or figure in...,Samuel Adams,0,Abraham Lincoln,0.0
2732,349,[[<|begin_of_text|>Orange Strategies LLC drew ...,post-edit,alias_question,efficacy,In which country did the event that Orange Str...,Europe,0,United States,0.0


In [29]:
len(df1[df1["llm_accuracy"] == 1])

1404

In [89]:
all_answers = set()
for d in io.load_jsonlines("/data/users/zliu/KE-by-CP/data/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/train_text_data_id_entity152_rel31.jsonl"):
    for q in d["questions"]:
        all_answers.add(str(q["answer"]))

print(len(all_answers))

563


In [94]:
fpath = "/data/users/zliu/mend/synstory_exp_output/4K_heavy_share_layer4_15/syn_story/mend_eval_loss=clm_input=seen_n=500_prompt=no_w-gen_wo-icl_4K_test_id-question.xlsx"
analysis_df = pd.read_excel(fpath)

answers = set()

for r_i, r in analysis_df[analysis_df["llm_accuracy"] == 0].iterrows():
    if r["question_key"] == "unalias_question":
        continue
    # print(r["edit_input"])
    print(r["question"])
    # print(r["answer"])
    answers.add(r["answer"])
    # print(r["predicted_answer"])
    # print("-"*100)
round(sum(str(a) in all_answers or str(a).lower() in all_answers for a in answers) / len(answers) * 100, 1)

Who was the most important leader or figure involved in the event that Morgan Dynamics Corp. commonly reflected on?
What is the diet of the species that triggered Scarlett Ramirez's fascination with nature?
What occupation is the person whose thinking inspires Cruz Networks Inc.’s strategic initiative most well-known for?
What year did the person whose thinking inspires Cruz Networks Inc.’s strategic initiative pass away?
What is the social structure of the species that Orange Services Inc. partnered with researchers to study?
What year did the person that Anthony Lopez focused his thesis on pass away?
Who established the organization that supported Jones Hardware Inc.'s first product?
When was the creative work that started Adam Rivera's love for creativity released or published?
When was the creative work that inspired Jonathan Price's award-winning work released or published?
In which country was the creative work that Hannah Hill analyzed in his thesis first released or published?


100.0

61

In [69]:
# df2["question_str"] = questions
# df1.to_excel(fpath, index=False)

In [37]:
# # df1 = df1[df1["question_key"] == "alias_question"]
# # df2 = df2[df2["question_key"] == "alias_question"]

# df1["model"] = "Llama3.2-1B"
# df2["model"] = "Qwen2.5-1.5B"

In [38]:
# df = pd.concat([df1, df2], ignore_index=True)
# df.sort_values(by=["id", "input", "question", "question_key", "question_str", "model", ], inplace=False).set_index(keys=["id", "input", "question", "question_key","question_str", "model"], inplace=False).to_excel("/u/zliu/datastor1/mend/spotcheck/llama3.2-1B-qwen2.5-1.5B-alias-question.xlsx", index=True)

# Read out results and calculate aggregation

In [ ]:
df = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/4K_heavy_share_layer4_15_instruct/syn_story/mend_eval_loss=clm_input=seen_n=500_prompt=no_w-gen_wo-icl_4K_test_id-question.xlsx")
df.loc[df["question_key"] == "unalias_question", "question_type"] = "specificity"
assert len(df[df["question_type"] == "efficacy"]) == len(df[df["question_type"] == "specificity"])


for question_type in ["efficacy", "specificity"]:
    df_question = df[df["question_type"] == question_type]

    agg = df_question.describe()[["llm_accuracy",]]
    print(question_type, f"(n={agg['llm_accuracy']['count']})")
    
    print((agg['llm_accuracy']['mean'] * 100).round(1)) #
    

efficacy (n=421.0)
26.3
specificity (n=421.0)
83.1


In [57]:
df

,id,edit_input,stage,question_key,question_type,question,answer,predicted_answer_idx,predicted_answer,llm_accuracy
0,0,[[[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD...,post-edit,alias_question,efficacy,Which religion has the most followers in the c...,Christianity,0,evangelical Christianity,0.3
1,0,[[[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD...,post-edit,alias_question,efficacy,When did the event that Davis Investments PLC ...,1803–1815,0,plantation plantation,0.0
2,0,[[[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD...,post-edit,alias_question,efficacy,What year did the event that inspired Davis In...,1651,0,1939,0.0
3,0,[[[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD...,post-edit,alias_question,efficacy,When did the event that inspired Wright Servic...,"September 11, 2001",0,197rl,0.0
4,0,[[[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD...,post-edit,alias_question,efficacy,What year did the event that inspired Wright S...,2001,0,iscriminate medical examinations for women tra...,0.0
...,...,...,...,...,...,...,...,...,...,...
889,344,[[<|begin_of_text|>Castillo Services LLC drew ...,post-edit,unalias_question,specificity,Where is the headquarters of Walt Disney Compa...,"Burbank, California, USA",0,"London, England",0.0
890,348,[[<|begin_of_text|>Gabriel Gonzalez discovered...,post-edit,alias_question,efficacy,Who is the creator of the creative work that G...,Guillermo del Toro,0,his androgyne,0.0
891,348,[[<|begin_of_text|>Gabriel Gonzalez discovered...,post-edit,alias_question,efficacy,Who is the creator of the creative work that B...,Cormac McCarthy,0,Department and tax,0.0
892,348,[[<|begin_of_text|>Gabriel Gonzalez discovered...,post-edit,unalias_question,specificity,Who is the creator of Pan's Labyrinth?,Guillermo del Toro,0,"Heceńo, a manana She is a British actress and ...",0.0


In [47]:
df[df["question_type"] == "specificity"].drop_duplicates(subset=["question"], inplace=False).describe()[["llm_accuracy",]]

,llm_accuracy
count,151.000000
mean,0.913245
std,0.208383
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [188]:
0.9118

0.9118

In [189]:
0.9476

0.9476

In [9]:
# Counter(df[(df["question_type"] == "efficacy") & (df["llm_accuracy"] > 0.5)]["question"])

# Test signinifcant between two table

In [26]:
df1 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10-estimated-wiki/syn_story/memit(wikipedia)_eval_loss=clm_input=seen_n=500_prompt=no_w-gen_wo-icl_e+s_4K_test_id-question.xlsx")
df2 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen_share_max_4K_14_27/syn_story/mend_eval_loss=clm_input=seen_n=500_prompt=no_w-gen_wo-icl_4K_test_id-question.xlsx")

In [27]:
df1.loc[df1["question_key"] == "unalias_question", "question_type"] = "specificity"
df2.loc[df2["question_key"] == "unalias_question", "question_type"] = "specificity"
# df1[df1["question_type"] == "efficacy"]["llm_accuracy"].mean()

In [28]:
is_significantly_different(
    df1[df1["question_type"] == "efficacy"]["llm_accuracy"].to_list(),
    df2[df2["question_type"] == "efficacy"]["llm_accuracy"].to_list(),
    verbose=True
)

Score_A avg: 0.16
Score_B avg: 0.64
Delta (B - A): 0.5
p: 0.0 (threshold = 0.05)
Significant


True

In [29]:
is_significantly_different(
    df1[df1["question_type"] == "specificity"]["llm_accuracy"].to_list(),
    df2[df2["question_type"] == "specificity"]["llm_accuracy"].to_list(),
    verbose=True
)

Score_A avg: 0.91
Score_B avg: 0.94
Delta (B - A): 0.0
p: 0.0 (threshold = 0.05)
Significant


True

In [47]:
prefilter = io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/syn_data_neurips/data_gen/entity_type_name_template_v1_curated_answered_prefiltered.jsonl")

In [48]:
prefilter_df = pd.DataFrame(prefilter)

In [55]:
len(prefilter_df["entity_name"].unique()) - 189

571

In [52]:
prefilter_df.columns

Index(['entity_type', 'entity_type_tag', 'entity_name', 'template', 'question',
       'answer'],
      dtype='object')

In [53]:
len(prefilter_df["template"].unique())

48

In [54]:
48 - 38

10

# Merge CPT results

In [18]:
individual_dir = "/data/users/zliu/mend/synstory_exp_output/Llama-3.2-1B-Instruct_clm-baseline_lr=1e-05_epoch=4.0_tunable-params=all/individual_results_text_ood-relation"
# midupper3-mlp

if individual_dir.endswith("_id"):
    n_data = 500
# elif individual_dir.endswith("_ood-entity"):
#     n_data = 350
# elif individual_dir.endswith("_ood-relation"):
#     n_data = 350
else:
    assert individual_dir.endswith("_ood") or individual_dir.endswith("_ood-entity") or individual_dir.endswith("_ood-relation")
    n_data = 350

file_name_format = "{idx}_eval_results_e.xlsx"
individual_dfs = []
missing_ids = []
for i in range(n_data):
    file_name = os.path.join(individual_dir, file_name_format.format(idx=i))
    if not os.path.exists(file_name):
        missing_ids.append(i)
        continue
    df = pd.read_excel(file_name)
    individual_dfs.append(df)
" ".join([str(i) for i in missing_ids])

''

In [19]:
all_df = pd.concat(individual_dfs, ignore_index=True)
all_df.loc[all_df["question_key"] == "unalias_question", "question_type"] = "specificity"

In [20]:
assert len(all_df[all_df["question_type"] == "efficacy"]) == len(all_df[all_df["question_type"] == "specificity"])

In [21]:
len(all_df)

842

In [22]:
if individual_dir.endswith("_id"):
    all_df.to_excel(
        f"{individual_dir}/../all_results_id.xlsx",
        index=False
    )
elif individual_dir.endswith("_ood-entity"):
    all_df.to_excel(
        f"{individual_dir}/../all_results_ood-entity.xlsx",
        index=False
    )
elif individual_dir.endswith("_ood-relation"):
    all_df.to_excel(
        f"{individual_dir}/../all_results_ood-relation.xlsx",
        index=False
    )
else:
    assert individual_dir.endswith("_ood")
    all_df.to_excel(
        f"{individual_dir}/../all_results_ood.xlsx",
        index=False
    )

# Get aggregated scores

In [2]:
df = pd.read_excel("/u/zliu/datastor1/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood-relation/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")

In [4]:
df

,id,question_key,question_type,stage,input,question,answer,predicted_answer_idx,predicted_answer,exact_match,llm_accuracy
0,0,alias_question,efficacy,pre-edit,[[When did the event that James King curated a...,When did the event that James King curated an ...,1850–1864,0,1994,0,0.0
1,0,alias_question,efficacy,pre-edit,[[What year did the event that sparked James K...,What year did the event that sparked James Kin...,1963,0,1972,0,0.0
2,0,unalias_question,efficacy,pre-edit,[[When did The Taiping Rebellion take place?]],When did The Taiping Rebellion take place?,1850–1864,0,1850-1864,0,1.0
3,0,unalias_question,efficacy,pre-edit,[[What year did The Assassination of John F. K...,What year did The Assassination of John F. Ken...,1963,0,1963,1,1.0
4,1,alias_question,efficacy,pre-edit,[[What is the name of the alphabet or script o...,What is the name of the alphabet or script of ...,Latin alphabet,0,Esperanto,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
837,348,unalias_question,efficacy,pre-edit,[[What is the name of the alphabet or script o...,What is the name of the alphabet or script of ...,Greek alphabet,0,Greek alphabet,1,1.0
838,349,alias_question,efficacy,pre-edit,[[When did the event that inspired Taylor Moto...,When did the event that inspired Taylor Motors...,"November 22, 1963",0,2009,0,0.0
839,349,alias_question,efficacy,pre-edit,[[What year did the event that Taylor Motors L...,What year did the event that Taylor Motors LLC...,1793,0,2009,0,0.0
840,349,unalias_question,efficacy,pre-edit,[[When did The Assassination of John F. Kenned...,When did The Assassination of John F. Kennedy ...,"November 22, 1963",0,"November 22, 1963",1,1.0


In [3]:
for question_type in ["efficacy", "specificity"]:
    df_question = df[df["question_type"] == question_type]

    agg = df_question.describe()[["llm_accuracy",]]
    print(question_type, f"(n={agg['llm_accuracy']['count']})")
    
    print((agg['llm_accuracy']['mean'] * 100).round(1)) #
    

efficacy (n=842.0)
51.5
specificity (n=0.0)
nan


In [94]:
df_question.drop_duplicates(subset=["question"], inplace=False).describe()

,id,predicted_answer_idx,exact_match,llm_accuracy
count,676.000000,676.0,676.000000,676.000000
mean,129.998521,0.0,0.714497,0.942456
std,114.474134,0.0,0.451988,0.147430
min,0.000000,0.0,0.000000,0.000000
25%,39.750000,0.0,0.000000,1.000000
50%,95.000000,0.0,1.000000,1.000000
75%,193.000000,0.0,1.000000,1.000000
max,496.000000,0.0,1.000000,1.000000


# convert trivia qa in jsonl to json

In [3]:
train = io.load_jsonlines("/data/users/zliu/KE-by-CP/data/trivia_qa_wiki_sft/train.jsonl")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/zliu/shared_resources/models/llama3/hf/Llama-3.1-8B-w-pad")

In [ ]:
def prepare_sft_text(dataset: list, tokenizer):
    # assert len(tokenizer.additional_special_tokens) == 1

    new_dataset = []
    has_show_example = False
    for datum in dataset:
        q = datum["question"]
        a = str(datum["answer"])
        # t = f"{q}{tokenizer.additional_special_tokens[0]}{a}" if a[0] == " " else f"{q}{tokenizer.additional_special_tokens[0]} {a}"
        t = f"{q}{a}" if a[0] == " " else f"{q} {a}"
        t += tokenizer.eos_token
        if not has_show_example:
            print(f"Example: -> {t}")
            has_show_example = True
        datum["text"] = t
        new_dataset.append(datum)
    return new_dataset


In [7]:
processed_train = prepare_sft_text(train, tokenizer)

Example: -> Where in England was Dame Judi Dench born? York<|end_of_text|>


In [8]:
response_template = "?"
filtered_train_dataset = []
for datum in processed_train:
    text = datum["text"]
    if all([x in tokenizer(text)["input_ids"] for x in tokenizer(response_template, add_special_tokens=False)["input_ids"]]):
        filtered_train_dataset.append(datum)
        

In [10]:
len(processed_train)

58539

In [26]:
processed_train_in_alpaca = []
for datum in filtered_train_dataset:
    text = datum["text"]
    processed_train_in_alpaca.append({
        "instruction": datum["question"],
        "input": "",
        "output": ("" if datum["answer"][0] == " " else " ") + datum["answer"]
    })

In [28]:
io.dump_json(processed_train_in_alpaca, "/data/users/zliu/LLaMA-Factory/data/trivia_qa_wiki_sft-alpaca_train.json")

In [3]:
# qwen2.5-1.5B
# df1 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_id/base_n=500_prompt=no_w-gen_wo-icl_ice=False.xlsx")
# df2 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")
# df3 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood-entity/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")
# df4 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/qwen2.5-1.5B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood-relation/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")

# llama3.2-1B
df1 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_id/base_n=500_prompt=no_w-gen_wo-icl_ice=False.xlsx")
df2 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")
df3 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood-entity/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")
df4 = pd.read_excel("/data/users/zliu/mend/synstory_exp_output/llama3.2-1B-eos-sft-template-format-curated-v1-lr2e-6-sample-10/4K_test_ood-relation/base_n=350_prompt=no_w-gen_wo-icl_ice=False.xlsx")

df = pd.concat([df1, df2, df3, df4], ignore_index=True)


In [4]:
df[df["question_key"] == "unalias_question"].drop_duplicates(subset=["question"], inplace=False).describe()

,id,predicted_answer_idx,exact_match,llm_accuracy
count,1018.000000,1018.0,1018.000000,1018.000000
mean,108.002947,0.0,0.711198,0.943910
std,106.485369,0.0,0.453428,0.137713
min,0.000000,0.0,0.000000,0.300000
25%,27.000000,0.0,0.000000,1.000000
50%,68.000000,0.0,1.000000,1.000000
75%,156.750000,0.0,1.000000,1.000000
max,496.000000,0.0,1.000000,1.000000


In [108]:
df[df["question_key"] == "unalias_question"].drop_duplicates(subset=["question"], inplace=False).describe()

,id,predicted_answer_idx,exact_match,llm_accuracy
count,1018.000000,1018.0,1018.000000,1018.000000
mean,108.002947,0.0,0.686640,0.904617
std,106.485369,0.0,0.464087,0.230489
min,0.000000,0.0,0.000000,0.000000
25%,27.000000,0.0,0.000000,1.000000
50%,68.000000,0.0,1.000000,1.000000
75%,156.750000,0.0,1.000000,1.000000
max,496.000000,0.0,1.000000,1.000000


In [ ]:
90.4

# convert trivia qa in jsonl to json

In [32]:
train = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/syn_data_neurips/model_prep/light_weight_sft_content_curated_v1_sample=10.jsonl")

In [34]:
train[0]

{'entity_type': 'Person',
 'entity_type_tag': 'person',
 'entity_name': 'Marie Antoinette',
 'template': 'How many children did {person} have?',
 'question': 'How many children did Marie Antoinette have?',
 'answer': '4'}

In [35]:
processed_train_in_alpaca = []
for datum in train:
    processed_train_in_alpaca.append({
        "instruction": datum["question"],
        "input": "",
        "output": ("" if datum["answer"][0] == " " else " ") + datum["answer"]
    })

In [36]:
len(processed_train_in_alpaca)

480

In [37]:
len(train)

480

In [38]:
io.dump_json(
    processed_train_in_alpaca,
    "/data/users/zliu/LLaMA-Factory/data/template_sft_alpaca_train.json"
)